In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [2]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [3]:
example = [
    HumanMessage(content="Add a task to submit the project report by Monday."),
    AIMessage(content="", additional_kwargs={
        "function_call": {
            "name": "add_task",
            "arguments": '{"title":"Submit the project report","deadline":"Monday"}'
        }
    }),
    ToolMessage(content='{"status": "success", "task": {"title": "Submit the project report", "deadline": "2023-11-13", "created_at": "2023-11-10 14:30:00"}}', tool_call_id="123"),
    
    HumanMessage(content="Remind me to call John at 3 PM with high priority."),
    AIMessage(content="", additional_kwargs={
        "function_call": {
            "name": "set_reminder",
            "arguments": '{"task_title":"Call John","reminder_time":"3 PM","priority":"high"}'
        }
    }),
    ToolMessage(content='{"status": "success", "reminder": {"task_title": "Call John", "reminder_time": "15:00", "priority": "high", "created_at": "2023-11-10 14:32:00"}}', tool_call_id="456"),
    
    HumanMessage(content="What tasks do I have today?"),
    AIMessage(content="", additional_kwargs={
        "function_call": {
            "name": "get_query",
            "arguments": '{"type":"tasks"}'
        }
    }),
    ToolMessage(content='{"tasks": [{"title": "Submit the project report", "deadline": "2023-11-13", "created_at": "2023-11-10 14:30:00"}]}', tool_call_id="789")
]

In [41]:
system_prompt = """You are a helpful task and reminder assistant. You have access to these tools:
- add_task: Adds a new task with a deadline
- set_reminder: Sets a reminder for a specific task
- get_query: Retrieves tasks or reminders

Always use the tools when appropriate to complete user requests and return the data as JSON format output."""

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("placeholder", "{example}"),
    ("human", "{input}")
])

In [18]:
tasks = []
reminders = []

In [ ]:
from typing_extensions import Annotated, BaseModel
class add_task(BaseModel):
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    title: Annotated[str,...,"Title of task."]
    deadline: Annotated[str,...,"Deadline of task."]

class set_reminder(BaseModel):
    """Sets a reminder for a specific task at a given time with priority (high, medium, low)."""
    task_title: Annotated[str,...,"Title of task."]
    reminder_time: Annotated[str,...,"Reminder Time."]
    priority:Annotated[str,...,"Priority of task."]="medium"

class get_query(BaseModel):
    type:Annotated[str,...,"Type of task."]

In [20]:
@tool
def add_task(title: str, deadline: str) -> Dict:
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    task = {
        "title": title,
        "deadline": deadline,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    tasks.append(task)
    return {"status": "success", "task": task}

@tool
def set_reminder(task_title: str, reminder_time: str, priority: str) -> Dict:
    """Sets a reminder for a specific task at a given time with priority (high, medium, low)."""
    reminder = {
        "task_title": task_title,
        "reminder_time": reminder_time,
        "priority": priority.lower(),
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    reminders.append(reminder)
    return {"status": "success", "reminder": reminder}

@tool
def get_query(type: str) -> Dict:
    """Retrieves tasks or reminders based on the user's query. Type can be 'tasks' or 'reminders'."""
    if type == "tasks":
        return {"tasks": tasks}
    elif type == "reminders":
        return {"reminders": reminders}
    else:
        return {"error": "Invalid query type"}


In [42]:
tools = [add_task, set_reminder, get_query]
# llm_with_tools = llm.bind_tools(tools)

In [43]:
agent = prompt | llm

In [44]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory)


In [30]:
# agent.invoke("Add a task to submit the project report by Monday.")
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Add a task to submit the project report by Monday.")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Add a task to submit the project report by Monday.
================================== Ai Message ==================================
Tool Calls:
  add_task (call_tqeHBWE0nDMPQhTmeiRZVh5o)
 Call ID: call_tqeHBWE0nDMPQhTmeiRZVh5o
  Args:
    title: Submit the project report
    deadline: Monday
================================= Tool Message =================================
Name: add_task

{"status": "success", "task": {"title": "Submit the project report", "deadline": "Monday", "created_at": "2025-04-15 16:02:07"}}
================================== Ai Message ==================================

I've added the task to submit the project report by Monday. If you need anything else, just let me know!


In [32]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats are the tasks for me today?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats are the tasks for me today?
================================== Ai Message ==================================
Tool Calls:
  get_query (call_xR6bgoAHdGpeuT8svcGlFEdI)
 Call ID: call_xR6bgoAHdGpeuT8svcGlFEdI
  Args:
    type: tasks
================================= Tool Message =================================
Name: get_query

{"tasks": [{"title": "Submit the project report", "deadline": "Monday", "created_at": "2025-04-15 16:02:07"}]}
================================== Ai Message ==================================

You have one task for today: 

- **Submit the project report** (Deadline: Monday)

If you need to add more tasks or set reminders, feel free to ask!


In [47]:
agent_executor.invoke(
    {"messages": [HumanMessage(content="is any reminder for me today on high priority?")]},
    config,
    stream_mode="values",
)["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yes, you have a high-priority reminder for "Cutting fruits" that is due today.
